In [157]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiment = SentimentIntensityAnalyzer()
import pandas as pd
import smtplib
from email.message import EmailMessage

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [158]:
#convert the sheets to dataframe
def convert_gsheets_url(u):
    try:
        worksheet_id = u.split('#gid=')[1]
    except:
        # Couldn't get worksheet id. Ignore it
        worksheet_id = None
    u = re.findall('https://docs.google.com/spreadsheets/d/.*?/',u)[0]
    u += 'export'
    u += '?format=csv'
    if worksheet_id:
        u += '&gid={}'.format(worksheet_id)
    return u

responses_sheet = 'https://docs.google.com/spreadsheets/d/1gnFd_M0PdtMoD7pGQxBwjtQPkanHKL_hsX0OrMgB0Ms/edit?usp=sharing'
try:
    url = convert_gsheets_url(responses_sheet)
    responses = pd.read_csv(url)
    print('Read successfully')
except:
    print(f"Could not read any data from the URL you provided.")

responses

Read successfully


,Timestamp,First Name,Last Name,Email,"How are you feeling today, in comparison to your usual state?",How are you usually compared to other people?,How have your activity levels changed over the past 30 days?,"Compared to other people, my level of activity energy and mood are ___ :",Have you experienced any significant changes in weight in the past 30 days?,How would you describe your sleep during this time?,How would you describe the changes in your moods? How fast or predictable are these changes?,Describe your level of risk taking during this time?,Are you more comfortable with meeting more people than usual?,"What changes, if any, in the amount of money you spend occur during this time?"
0,2/19/2023 16:44:55,archita,khaire,archita.khaire@gmail.com,I feel lonely depressed sad suicidal and disin...,I am usually more lonely and misunderstood. I ...,I have been lethargic.,introvert.,I have been skinnier.,restlessness.,I have always been depressed. no changes in my...,I am reserved.,no I hate people I am an introvert.,I don't spend my money at all.
1,2/19/2023 16:47:46,Isha,Mahadeshwar,isha02m@gmail.com,I feel ok. I am more emotional than usual. I f...,I am happy and sad. I am fun. I like to be lau...,I feel more energy.,i am high energy.,my weight has not changed.,i sleep less. I,I have mood swings. I am moody.,I am very impulsive.,"Yes, I love meeting people. I am an extrovert.",I love to go shopping.


In [159]:
responses_full_list = []
for i in range(0,len(responses)):
    indv_response = responses.loc[i].values.tolist()
    merged_indv_response = ' '.join(indv_response[4 : 13])
    responses_full_list.append(merged_indv_response)

responses_full_list

['I feel lonely depressed sad suicidal and disinterested in general life. I am usually more lonely and misunderstood. I feel like an introvert. I have been lethargic. introvert. I have been skinnier.  restlessness. I have always been depressed. no changes in my mood very often. I am reserved.  no I hate people I am an introvert.',
 'I feel ok. I am more emotional than usual. I feel like doing spontaneous things more. I am happy and sad. I am fun. I like to be laughing. I like to shop. I feel more energy. i am high energy. my weight has not changed. i sleep less. I  I have mood swings. I am moody. I am very impulsive. Yes, I love meeting people. I am an extrovert.']

In [160]:
depression_symptoms = ["skinnier","lazy","tired","disinterested","worthless","suicidal","fatter","bigger","satiated","full","emptiness","angry","anxiety", "slow","outbursts","frustration","sadness","teary","hopeless","troubled","pain","headaches","guilt","blame","slow","craving","restlessness","misunderstood","negative","introvert","aching","lethargic","reserved"]
bipolar_symptoms = ["sleeplessness","gambling","reckless","moody","irritated","sad","hopeless","happy","talkative","distracted","energetic","confident","fatigue","energetic","impulsive","shopping","expensive","excited","racing","obsessive","narcissist","sleepy","inactive","elated","egotistic","high","smug","ego","spontaneous","passionate","sensual"]
print(len(depression_symptoms))
print(len(bipolar_symptoms))

33
31


In [161]:
def generatepred(indv_response):
    stop_words = set(stopwords.words('english'))
    my_tokenizer = RegexpTokenizer(r'\w+')
    tokens = my_tokenizer.tokenize(indv_response)
    filtered_sentence = [w for w in tokens if not w.lower() in stop_words]


    sentiment_list = []
    for i in filtered_sentence:
        pol = [sentiment.polarity_scores(i), i]
        sentiment_list.append(pol)

    filt_sentiment_list = []
    filt_bysentiment_sentence = []
    for i in sentiment_list:
        if i[0]['compound'] != 0.0:
            filt_sentiment_list.append(i[0])
            filt_bysentiment_sentence.append(i[1])

    sum_sentiment = 0
    for i in filt_sentiment_list:
        sum_sentiment += i['compound']
    avg_sentiment = (sum_sentiment)/(len(filt_sentiment_list))


    depression_exactmatch_count = 0
    bipolar_exactmatch_count = 0

    for i in depression_symptoms:
        for j in filt_bysentiment_sentence:
            if i == j:
                depression_exactmatch_count += 1

    for i in bipolar_symptoms:
        for j in filt_bysentiment_sentence:
            if i == j:
                bipolar_exactmatch_count += 1

    synonyms_full = []
    for j in filt_bysentiment_sentence:
        for syn in wordnet.synsets(j):
            for l in syn.lemmas():
                synonyms_full.append(l.name())

    def unique(list1):
        unique_list = []
        for x in list1:
            if x not in unique_list:
                unique_list.append(x)
        return unique_list

    filtsentence_synonyms = unique(synonyms_full)

    depression_synmatch_count = 0
    bipolar_synmatch_count = 0

    for i in depression_symptoms:
        for j in filtsentence_synonyms:
            if i == j and j not in filt_bysentiment_sentence:
                depression_synmatch_count += 1

    for i in bipolar_symptoms:
        for j in filtsentence_synonyms:
            if i == j and j not in filt_bysentiment_sentence:
                bipolar_synmatch_count += 1

    depression_exactsyn_sum = depression_exactmatch_count + depression_synmatch_count
    bipolar_exactsyn_sum = bipolar_exactmatch_count + bipolar_synmatch_count

    if depression_exactsyn_sum > bipolar_exactsyn_sum and avg_sentiment <= 0:
        return "The results of our questionnaire indicate that you have a low vulnerability to bipolar disorder at this time. General mental health resources are provided on our website. Please refer to the 'depression resources' and 'mental health resources' tabs for information catered to your specific demographic."

    if bipolar_exactsyn_sum > depression_exactsyn_sum and avg_sentiment > 0:
        return "The results of our questionnaire indicate that you have a high vulnerability to bipolar disorder at this time. General mental health resources are provided on our website. Please refer to the 'bipolar disorder resources' and 'mental health resources' tabs for information catered to your specific demographic."

    if depression_exactsyn_sum > bipolar_exactsyn_sum and avg_sentiment > 0:
        return "The results of our questionnaire indicate that you have a moderate vulnerability to bipolar disorder at this time. General mental health resources are provided on our website. Please refer to the 'bipolar disorder resources' and 'mental health resources' tabs for information catered to your specific demographic."

    if bipolar_exactsyn_sum > depression_exactsyn_sum and avg_sentiment <= 0:
        return "The results of our questionnaire indicate that you have a moderate vulnerability to bipolar disorder at this time. General mental health resources are provided on our website. Please refer to the 'bipolar disorder resources' and 'mental health resources' tabs for information catered to your specific demographic."

    if bipolar_exactsyn_sum == depression_exactsyn_sum:
        if avg_sentiment > 0:
            return "The results of our questionnaire indicate that you have a moderate vulnerability to bipolar disorder at this time. General mental health resources are provided on our website. Please refer to the 'bipolar disorder resources' and 'mental health resources' tabs for information catered to your specific demographic."
        if avg_sentiment <= 0:
            return "The results of our questionnaire indicate that you have a low vulnerability to bipolar disorder at this time. General mental health resources are provided on our website. Please refer to the 'depression resources' and 'mental health resources' tabs for information catered to your specific demographic."


In [162]:
index = -1
emails_results = []
for res in responses_full_list:
    index += 1
    indv_result = []
    email = str(responses.iloc[index,3])
    indv_result.append(email)
    indv_result.append(generatepred(res))
    emails_results.append(indv_result)

emails_results

[['archita.khaire@gmail.com',
  "The results of our questionnaire indicate that you have a low vulnerability to bipolar disorder at this time. General mental health resources are provided on our website. Please refer to the 'depression resources' and 'mental health resources' tabs for information catered to your specific demographic."],
 ['isha02m@gmail.com',
  "The results of our questionnaire indicate that you have a high vulnerability to bipolar disorder at this time. General mental health resources are provided on our website. Please refer to the 'bipolar disorder resources' and 'mental health resources' tabs for information catered to your specific demographic."]]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8bcf98c6-da23-40b1-ad73-ad39c5cc1697' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>